In [2]:
import pandas as pd
data = pd.read_csv('./预处理过的论坛数据.csv')
#data['summarization'] = 'N'
data.head()

,content,publish_time,view_counts,text_length,summarization
0,金价格中线看涨的因素还在待到债务危机避险情绪的修正完毕后经济增速放缓的美国以及削减赤字对欧洲...,2010-07-21 16:58:34,1244,361,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...
1,从日K线来看 黄金处于交易日大幅走低跌破5日均线后大幅走低至200日均线即1160附近获得支...,2010-07-30 11:47:39,1863,385,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑
2,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,2010-08-04 07:48:00,7636,78,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...
3,美元反弹让昨天盯住美元的我们在黄金上获得了良好的收益继续扩大的美元反弹空间将会终结黄金7月底...,2010-08-10 14:38:00,1086,405,周三(1月1日)是全球金融系统危机以来最严重的一次跌破1 31的跌势。
4,行情回顾美联储购国债刺激经济致美股暴跌引发短期避险大力提振美元指数受此影响金银纷纷下跌美盘前...,2010-08-12 11:33:18,1558,461,美联储购国债刺激经济致美股暴跌引发短期避险大力提振美元指数受此影响金银纷纷下跌。


In [8]:
data.loc[2999]

content          各国央行都是假设自己是理性的8月这波避险情节的大散户博傻多头行情结束了美元本位体系是严重动摇...
publish_time                                   2011-09-24 02:04:11
view_counts                                                   2796
text_length                                                    105
summarization                 国际货币基金组织(IMF)今年8月在瑞士布鲁塞尔推出了一次“黄金周” 。
Name: 2999, dtype: object

In [6]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
WHITESPACE_HANDLER = lambda k: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', k.strip()))
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False, clean_up_tokenization_spaces=True, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def summarize_text(text):
    # 预处理文本
    input_ids = tokenizer(
        [WHITESPACE_HANDLER(text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=1024
    )["input_ids"]
    
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary

In [6]:
from tqdm import tqdm
for i in tqdm(range(0, 1000), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('./预处理过的论坛数据.csv',index=False)
print(data.iloc[0:100])

Processing rows: 100%|███████████████████████████████████████████████████████████| 1000/1000 [3:04:17<00:00, 11.06s/it]


                                              content         publish_time  \
0   金价格中线看涨的因素还在待到债务危机避险情绪的修正完毕后经济增速放缓的美国以及削减赤字对欧洲...  2010-07-21 16:58:34   
1   从日K线来看 黄金处于交易日大幅走低跌破5日均线后大幅走低至200日均线即1160附近获得支...  2010-07-30 11:47:39   
2   黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...  2010-08-04 07:48:00   
3   美元反弹让昨天盯住美元的我们在黄金上获得了良好的收益继续扩大的美元反弹空间将会终结黄金7月底...  2010-08-10 14:38:00   
4   行情回顾美联储购国债刺激经济致美股暴跌引发短期避险大力提振美元指数受此影响金银纷纷下跌美盘前...  2010-08-12 11:33:18   
..                                                ...                  ...   
95  BEST黄金现货金1350一道门槛 二十国集团G20誓言避免货币竞争性贬值并同意给予新兴经济...  2010-10-25 15:12:42   
96  当前金价面临诸多避险买盘以及投机盘的推动此外黄金正处于实物旺季消费的当口这也将继续构成中期黄...  2010-10-25 16:45:28   
97  金骏点金 除非你是基本面分析大师否则基本面信息会使你昏头转向混淆视听在你对基本面把握能力不够...  2010-10-26 09:10:34   
98  SPDR持仓量不变保持1 298 27吨市场对美联储量化宽松策的预期盖过了G20影响早盘金价...  2010-10-26 10:30:52   
99  早在G20峰会之前黄金已提前反映下跌因此G20峰会后反而利空出尽金价反弹 深层原因是10月份...  2010-10-27 11:43:01   

   view_counts  text_length                                    

In [17]:
from tqdm import tqdm
for i in tqdm(range(1000, 1500), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('./预处理过的论坛数据.csv',index=False)
print(data.iloc[1000:1500])

Processing rows: 100%|█████████████████████████████████████████████████████████████| 500/500 [1:44:44<00:00, 12.57s/it]


                                                content         publish_time  \
1000  金价周五下跌 因美元兑欧元上涨及欧元区债务问题的不确定性再度浮现 促使周末前抛售黄金 银价持...  2011-05-16 11:11:19   
1001  黄金从周线上我们可以看出本周收出一颗十字星基本来讲十字星有两种意义中继和反转在上周强势下跌的...  2011-05-16 11:13:49   
1002  基本面分析 美联储主席伯南克 表明美联储打算在相当长时间内维持利率于纪录低位 在6月末结束6...  2011-05-16 11:33:23   
1003  金评在线 上海黄金TD上周从318的高位跳水至今日现价313这说明近日金价卖盘量大幅增加促使...  2011-05-16 16:41:32   
1004  操作建议黄金因为在周五1480又再次受到支撑上方阻力关注14981503短线交易者可参考上图...  2011-05-16 16:48:55   
...                                                 ...                  ...   
1495  昨天白银开盘顺势而下后在反弹原油几天的连续大的跌幅昨天在90美元附近才刚刚开始有所稳定所以今...  2011-06-28 10:06:38   
1496  周一金价维持小幅震荡格局最高1505最低1491日K线收小十字星金价自上周以来连续多日下跌主...  2011-06-28 10:09:31   
1497  昨天现货黄金延续跌势主要是有消息称欧盟正制定应急计划以防希腊议会拒绝通过紧缩计划从而导致希腊...  2011-06-28 10:55:06   
1498  刚哥点金628黄金行情分析 行情回顾昨日金价最够触及1504最低触及1490全天波幅14美金...  2011-06-28 11:53:24   
1499  金生金工作室 1500美元位置金价何去何从市场调整继续 但要注意逐渐止盈 周一贵金属延续上周...  2011-06-28 13:12:37   

     view_counts  text_length  \
1000  

In [10]:
#data = pd.read_csv('./预处理过的论坛数据.csv')
data.loc[2350]

content          沃伦点金 黄金投资中捡钱的方法 很多因为交易而困惑很多人因为心态而烦恼告诉你下面5种捡钱的方...
publish_time                                   2011-08-24 09:36:33
view_counts                                                    781
text_length                                                    510
summarization                                                    N
Name: 2350, dtype: object

In [12]:
from tqdm import tqdm
for i in tqdm(range(1500, 3000), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('./预处理过的论坛数据.csv',index=False)
print(data.iloc[2900:3000])

Processing rows: 100%|███████████████████████████████████████████████████████████| 1500/1500 [4:51:49<00:00, 11.67s/it]


                                                content         publish_time  \
2900                 世界站在一个关键时刻在新机会出现之前黄金天然金融属性的复辟就不会结束  2011-09-19 23:45:19   
2901  国家投入巨资支持新药研发特别关注一股抗癌新品 赤天化正与贵州大学合作开发研究提取竹叶中的黄酮...  2011-09-20 08:36:46   
2902  行情回顾昨日受美元重压现货金收低盘间高及1 826 79美元 盎司低探至1 769 59美元...  2011-09-20 09:03:34   
2903  昨晚间黄金的行情在最低点1770 03后就一直在低位横盘走势最高上扬到1784 43今天没有...  2011-09-20 09:23:43   
2904  纽约黄金期货价格周一收盘下跌至三个多星期以来的最低水平原因是美国股票市场和商品期货市场均有所...  2011-09-20 09:34:31   
...                                                 ...                  ...   
2995  老美扭曲购买债券计划也是拖延时间而已美元的宿命并没改变欧洲股市跌得厉害有机构卖黄金白银对冲黄...  2011-09-23 22:57:33   
2996        一旦对经济又有期望股市见底后全球资金又将反抢已经惨跌的股市很可能雪上加霜黄金被彻底抛弃  2011-09-23 23:12:03   
2997  昨天晚间写了做内盘聚乙烯现在想来都知道后果了既然敢使用20的杠杆在敞口中放手博弈也就作好了倾...  2011-09-24 00:40:09   
2998   很遗憾我是愁的那一个只能自我安慰下浅入浅出总比满满爆仓的好黄金已经疯狂的没有支撑没有底位了只有跌  2011-09-24 01:15:38   
2999  各国央行都是假设自己是理性的8月这波避险情节的大散户博傻多头行情结束了美元本位体系是严重动摇...  2011-09-24 02:04:11   

     view_counts  text_length  \
2900  

In [14]:
data_3000 = data.iloc[:3000]

In [12]:
data_3000=pd.read_csv('./预处理过的3000条论坛数据.csv')

In [14]:
data_3000['text_length'] = data_3000['summarization'].astype(str).apply(len)
data_3000 = data_3000[(data_3000['text_length'] >= 10)]
data_3000 = data_3000[data_3000['summarization'].str.contains('黄金|金价')]
data_3000 = data_3000.drop(columns=['content', 'text_length'])
data_3000 = data_3000.reset_index(drop=True)
data_3000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2035 entries, 0 to 2034
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_time   2035 non-null   object
 1   view_counts    2035 non-null   object
 2   summarization  2035 non-null   object
dtypes: object(3)
memory usage: 47.8+ KB


In [34]:
column_data = data_3000['summarization']
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
column_data[990:1000]

990                                                                       黄金白银周五(8月20日)继续下跌
991    国际能源署突然宣布未来两个月将释放6000万桶紧急原油储备引发油价大幅下挫金融市场上投资者下调通胀预期打压高收益货币金价银价亦大幅下跌美元跌破100日均线后再次拉升日线
992                                                                          黄金专家级资金管理和交易策略
993                                      24日晚间黄金白银行情简评今日国际金价反弹至 152受阻欧盘时段回测15得随后再度反弹到 152水平
994                                                          今天受经济基本面的不利影响黄金白银原油等受压出现了不小的跌幅
995                                                                        黄金对抗通涨的最理想武器是什么?
996                              中国黄金周五(1月1日)开盘后出现重大的跌幅,黄金白银当天开盤后冲高至34 40或34 60附近遇阻力回落操作策略。
997                                          国际金价上周冲高至155后一路回落周五晚间最低至1500已达60美金周线收取带长上引线的阴柱
998                                      伦敦金 国际金价上周冲高至155后一路回落周五晚间最低至1500已达60美金周线收取带长上引线的阴柱
999                                                 中国江苏扬州黄金日贵金属经营有限公司专业从事黄金白银及现货延期交收电子交易业务
Name: summarization, dtype: object

In [30]:
data_3000['x'] = 'N'

In [36]:
X = pd.read_excel('./论坛数据标签1000条.xlsx')
X

,A,B
0,1,1
1,1,1
2,0,0
3,0,1
4,1,1
5,0,1
6,0,0
7,0,0
8,0,0
9,0,1


In [42]:
for i in range(500,1000):
    data_3000.loc[i,'X'] = X.loc[i-500,'B']

In [44]:
data_3000.to_csv('./预处理过的3000条论坛数据.csv')

In [10]:
from tqdm import tqdm
for i in tqdm(range(3000, 4500), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('./预处理过的论坛数据.csv',index=False)
print(data.iloc[3900:3950])

Processing rows: 100%|███████████████████████████████████████████████████████████| 1500/1500 [4:29:57<00:00, 10.80s/it]


                                                content         publish_time  \
3900  昨天黄金的走势比预期的弱早盘1783附近就急转而下美盘更是打破了1760的强支撑昨天总体在1...  2011-11-17 08:22:22   
3901  据美国商品期货交易委员会公布的最新数据显示黄金再次吸引了投机性交易商这类参与者以买家的身份重...  2011-11-17 09:17:14   
3902  炒金 一金价波动大根据国际黄金市场行情按照国际惯例进行报价因受国际上各种政治经济因素以及种突...  2011-11-17 09:42:05   
3903  目前情况来看欧债危机难以解决并且还在蔓延法国状况引起市场担忧前景不被市场看好不容乐观意大利新...  2011-11-17 09:42:41   
3904  基本面消息 美国劳工部昨日16日公布的报告显示在能源及汽车等制成品价格下降的影响下美国10月...  2011-11-17 09:51:54   
3905  基本面消息 美国劳工部昨日16日公布的报告显示在能源及汽车等制成品价格下降的影响下美国10月...  2011-11-17 09:56:34   
3906  基本面英国低迷的就业数据加剧了市场对英国经济基本面的担忧美国10月工业产出月率增长0 7预期...  2011-11-17 09:57:00   
3907  周三11月16日行情回顾美元指数延续涨势现货黄金周三承压收跌最近黄金与美元的反向联动关系表现...  2011-11-17 10:42:33   
3908          黄金目前1757一线支撑反弹可以逢低做多上方关注1765压力突破看17731775  2011-11-17 11:28:54   
3909  我知道很多人都很恐慌都很迷茫本来没有时间来写这些会对自己产生影响但这份利益对比多数人的心情又...  2011-11-17 11:51:43   
3910  金市策略 改革遇阻 利空加剧 欧债国家问题持续恶化令最近行情走势较弱昨日黄金开盘后依然维持惯...  2011-11-17 12:33:30   
3911  上一个交易日黄金回撤1778后震荡下跌日线收盘带下影线的阴线上一交易

In [14]:
data_1500 = data.iloc[3000:4500]

In [16]:
data_1500['text_length'] = data_1500['summarization'].astype(str).apply(len)
data_1500 = data_1500[(data_1500['text_length'] >= 10)]
data_1500 = data_1500[data_1500['summarization'].str.contains('黄金|金价')]
data_1500 = data_1500.drop(columns=['content', 'text_length'])
data_1500 = data_1500.reset_index(drop=True)
data_1500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_time   960 non-null    object
 1   view_counts    960 non-null    object
 2   summarization  960 non-null    object
dtypes: object(3)
memory usage: 22.6+ KB


C:\Users\86155\AppData\Local\Temp\ipykernel_28420\232988499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1500['text_length'] = data_1500['summarization'].astype(str).apply(len)


In [18]:
data_1500.to_csv('./预处理过的1500条论坛数据.csv',index=False)

In [2]:
import pandas as pd
data = pd.read_csv('./completedata9_13.csv')
data

,content,publish_time,view_counts,text_length,summarization
0,8月13日黄金操作建议多单操作建议一白盘16161618一线轻仓5多单进场止损3美金目标16...,2012-08-13 11:39:28,406,226,8月13日黄金操作建议一白盘价格首次触及1620附近多单进场10仓位目标161620
1,锐金团队 浮光跃金黄金交易必懂基础知识1 平均线 如果您相信技术分析中趋势是您的朋友的信条那...,2012-08-13 12:05:44,453,909,在经历了近20年金融危机之后 汇率日趋低迷 虽然投资者们对市场表现的信心有所忧虑但是对于一些...
2,国际金价近日窄幅波动涨跌互现尽管股市和原油等外围市场对黄金交易构成利好但仍不足以支撑金价上行...,2012-08-13 13:42:07,1476,741,国际金价近日窄幅波动涨跌互现尽管股市和原油等外围市场对黄金交易构成利好但仍不足以支持金价格上行
3,现货黄金价格在周一8月13日亚洲时段小幅走高但行情波动有限原因是市场对黄金的实物需求受到遏制...,2012-08-13 13:44:15,1372,203,现货黄金价格在周一(8月13日)亚洲时段小幅走高但行情波动有限原因是市场对黄金实物需求受到遏制
4,上周欧洲央行行长德拉基未能在上周的利率会议上兑现自己购债的承诺令市场大感失望不过本周有两位欧...,2012-08-13 13:46:13,3262,584,上周欧洲央行行长德拉基未能在上周的利率会议上兑现自己购债的承诺令市场大感失望
...,...,...,...,...,...
3995,国际现货黄金震荡中小幅走高白银亦录得一定涨幅由于下周一美国国债将触及上限市场情绪趋于谨慎预计...,2012-12-28 13:54:53,579,678,国际现货黄金震荡中小幅走高白银亦录得一定涨幅由于下周一美国国债将触及上限市场情绪趋于谨慎
3996,昨日回顾与今日展望 黄金昨日继续震荡白银则震幅加大黄金最低回踩至1652后回升但仍然在166...,2012-12-28 13:58:54,383,758,美国与共和党两党周四(11月29日)在众议院的谈判中达成协议时间可能不够而美股开始加速下跌跌...
3997,尽管圣诞节假期已过去两天但由于年底逼近且财政悬崖谈判仍旧迷雾重重因此部分投资者继续处于观望态...,2012-12-28 14:03:55,336,806,尽管圣诞节假期已过去两天但由于年底逼近且财政悬崖谈判仍旧迷雾重重因此部分投资者继续处于观望态...
3998,基本面分析 财政悬崖重启谈判美国两党互相表态美国参议院民主党领袖里德表示看来我们是要跌落财政...,2012-12-28 14:11:56,771,725,美国白宫表示总统奥巴马周五将与参议院民主党就财政悬崖重启谈判而金银价格快速震荡收小阳线


In [10]:
data

,content,publish_time,view_counts,text_length,summarization
0,金价格中线看涨的因素还在待到债务危机避险情绪的修正完毕后经济增速放缓的美国以及削减赤字对欧洲...,2010-07-21 16:58:34,1244,361,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...
1,从日K线来看 黄金处于交易日大幅走低跌破5日均线后大幅走低至200日均线即1160附近获得支...,2010-07-30 11:47:39,1863,385,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑
2,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,2010-08-04 07:48:00,7636,78,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...
3,美元反弹让昨天盯住美元的我们在黄金上获得了良好的收益继续扩大的美元反弹空间将会终结黄金7月底...,2010-08-10 14:38:00,1086,405,周三(1月1日)是全球金融系统危机以来最严重的一次跌破1 31的跌势。
4,行情回顾美联储购国债刺激经济致美股暴跌引发短期避险大力提振美元指数受此影响金银纷纷下跌美盘前...,2010-08-12 11:33:18,1558,461,美联储购国债刺激经济致美股暴跌引发短期避险大力提振美元指数受此影响金银纷纷下跌。
...,...,...,...,...,...
167842,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,2024-08-10 00:42:22,85,30,N
167843,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,2024-08-10 02:23:27,125,58,N
167844,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,2024-08-10 15:14:49,313,51,N
167845,8月9日 周五总结一下昨天单子操作5单止盈5单5连胜我们做投资理财本身就是要减少我们的风险把...,2024-08-10 20:01:52,180,158,N
